# CL3 - Intro to CNNs in Keras

Import necessary modules

In [ ]:
# The dataset
from keras.datasets import mnist

# Building the model
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D

# One-hot encoding
from keras.utils import np_utils

# Callbacks for training
from keras.callbacks import TensorBoard, EarlyStopping

# Ploting
import matplotlib.pyplot as plt
%matplotlib inline

# Ndarray computations
import numpy as np

### 1. Loading and visualizing the data

Load the data intro training and test sets.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Check range.

In [ ]:
print('Min:', X_train.min(), '\nMax:', X_train.max())

Check dimensions.

In [ ]:
X_train.shape

Take a look at one example.

In [ ]:
X_train[10]

Plot one example.

In [ ]:
plt.imshow(X_train[10], cmap='gray')

In [ ]:
y_train[10]

### 2. Preprocessing

Reshape the input dimensions to be `[samples][width][height][channels]`

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [ ]:
X_train.shape

Normalize inputs to 0-1.

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

One hot-encode output.

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
y_train[10]

### 3. Training

Create a tentative model.

In [ ]:
def base_model():
    
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Train it using Keras.

In [ ]:
# build the model
model = base_model()

# Fit the model
tb = TensorBoard(log_dir='./logs/initial_setting')
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=1024, callbacks=[tb])

Try different architectures.

In [ ]:
def more_layers_model():
    
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def more_filters_model():
    
    model = Sequential()
    model.add(Conv2D(100, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def more_neurons_model():
    
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def bnorm_model():
    
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Train all of them and compare the results using TensorBoard.

In [ ]:
model_names = ['more_layers_model', 'more_filters_model', 'more_neurons_model', 'bnorm_model']

for name in model_names:
    print('Training model:',name)
    model = globals()[name]()
    tb = TensorBoard(log_dir='./logs/'+name)
    model.fit(X_train,
              y_train, 
              validation_data=(X_test, y_test),
              epochs=15, 
              batch_size=1024, 
              callbacks=[tb],
              verbose=0)
    print('Done!')

Choose the best one and train longer. Here we also use [early stopping](https://en.wikipedia.org/wiki/Early_stopping#Validation-based_early_stopping).

In [ ]:
model = more_filters_model()

# Set callbacks
tb = TensorBoard(log_dir='./logs/final_model')
estop = EarlyStopping(monitor='val_acc', patience=5)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=1024, callbacks=[tb, estop])

### 4. Assessment

Evaluate accuracy on test set.

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy: %.2f%%" % (scores[1]*100))